In [2]:
import pandas as pd
import pycountry
import country_converter

## Chart 1: Maternaly Mortality vs GDP PC PPP

a helper function for country matching:

In [3]:
def ISO3(country_name):
    matches = { ## Hardcoding Strings pycountry won't catch
        'Bolivia (Plurinational State of)':'BOL',
        'Democratic Republic of the Congo':'COD',
        'Dem. Rep. Congo': 'COD',
        'Iran (Islamic Republic of)':'IRN',
        'Micronesia (Federated States of)':'FSM',
        'Türkiye': 'TUR',
        'TÃ¼rkiye': 'TUR',
        'Venezuela (Bolivarian Republic of)':'VEN',
        'Hong Kong SAR, China' : 'HKG',
        'SÃ£o TomÃ© and Principe' : 'STP',
        'Swaziland' : 'SWZ',
        'Laos' : 'LAO',
        'Lao PDR' : 'LAO',
        'Cape Verde' : 'CPV',
        "CÃ´te d'Ivoire": 'CIV'
    }
    if country_name in matches:
        return matches[country_name]
    try:
        return country_converter.convert(names=country_name)
    except:
        print(f'No Match for {country_name}')
        return None

In [5]:
def name_Flag(ISO3):
    country = pycountry.countries.get(alpha_3=ISO3)
    return(f'{country.name} {country.flag}')

In [6]:
def ISO3_flag(ISO3):
    country = pycountry.countries.get(alpha_3=ISO3)
    return(f'{country.flag}{ISO3}')

In [7]:
mm_df = pd.read_excel("Data 2023-01-02 15-07.xlsx", sheet_name="Data")
mm_df = mm_df[['Year', 'Country', 'WHO region', 'Value Numeric']]
mm_df['ISO3'] = mm_df.apply(lambda row: ISO3(row.Country) , axis = 1)
mm_df

,Year,Country,WHO region,Value Numeric,ISO3
0,2000,Afghanistan,Eastern Mediterranean,1450,AFG
1,2001,Afghanistan,Eastern Mediterranean,1390,AFG
2,2002,Afghanistan,Eastern Mediterranean,1300,AFG
3,2003,Afghanistan,Eastern Mediterranean,1240,AFG
4,2004,Afghanistan,Eastern Mediterranean,1180,AFG
...,...,...,...,...,...
3289,2013,Zimbabwe,Africa,509,ZWE
3290,2014,Zimbabwe,Africa,494,ZWE
3291,2015,Zimbabwe,Africa,480,ZWE
3292,2016,Zimbabwe,Africa,468,ZWE


In [8]:
pop_df = pd.read_excel("WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx", skiprows=16)
pop_df = pop_df[['Region, subregion, country or area *', 'ISO3 Alpha-code','Year', 'Total Population, as of 1 July (thousands)']]
pop_df.columns = ['Country', 'ISO3', 'Year', 'Population']

In [9]:
gdp_df = pd.read_csv('UNdata_Export_20230102_161208339.csv')
gdp_df['ISO3'] = gdp_df.apply(lambda row: ISO3(row['Country or Area']) , axis = 1)

Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Eastern and Southern not found in regex
Africa Easter

In [10]:
df = pd.merge(left=mm_df,right=gdp_df, left_on=['ISO3','Year'], right_on=['ISO3','Year'], how='inner')
df = df[['ISO3', 'Country', 'Year', 'Value', 'Value Numeric', 'WHO region']]
df.columns = ['ISO3', 'Country', 'Year', 'GDP pc', 'Maternal Mortality', 'Region']

df = pd.merge(df, pop_df, how='inner', left_on=['ISO3','Year'], right_on=['ISO3','Year'])
df = df[['Year', 'ISO3', 'Country_x', 'Region', 'GDP pc', 'Maternal Mortality', 'Population']]
df.columns = ['Year', 'ISO3', 'Country', 'Region', 'GDP pc', 'Maternal Mortality', 'Population']

it'd be nice to have flags with the names:

In [11]:
countries_with_flags = dict(zip(list(set(df.ISO3)),[name_Flag(ISO3) for ISO3 in set(df.ISO3)])) #Avoiding minutes of repeat calls
ISO3s_with_flags = dict(zip(list(set(df.ISO3)),[ISO3_flag(ISO3) for ISO3 in set(df.ISO3)]))
df['Country'] = df.apply(lambda row: countries_with_flags[row.ISO3] , axis = 1)
df['ISO3_flag']  = df.apply(lambda row: ISO3s_with_flags[row.ISO3] , axis = 1)
df = df.drop_duplicates(['Year', 'ISO3'])

In [12]:
with open('mortality_gdp.json', 'w') as f:
    f.write(df.to_json(orient='records'))

In [13]:
df.to_excel('processed.xlsx')

In [14]:
changes_df = df.loc[(df.Year==2000) | (df.Year==2017)][['Year', 'Country', 'ISO3', 'Maternal Mortality']]
changes_df = changes_df.drop_duplicates(['Year', 'ISO3'], False)
changes_df = changes_df.pivot(index="ISO3", columns=["Year"], values=["Maternal Mortality", 'Country'])


/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_9822/2460847052.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop_duplicates except for the argument 'subset' will be keyword-only.
  changes_df = changes_df.drop_duplicates(['Year', 'ISO3'], False)


In [15]:
changes_df.to_excel('changes.xlsx')

### Second Chart - Healthcare Expenditure vs Life Expectency

In [16]:
ex_df = pd.read_excel("expend.xlsx", skiprows=6)
ex_df = ex_df.iloc[1:39, [0]+list(range(3,len(ex_df.columns)))]
ex_df = ex_df.rename(columns={'Year':'Country'})


In [17]:
ex_df

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,Australia,3593.395,3809.112,3854.19,4087.849,4562.73,4777.388,5037.334,5075.384,5193.845,5130.234,5627.315,..
2,Austria,4258.99,4345.158,4588.344,4767.083,4858.332,4943.516,5195.532,5315.355,5518.538,5624.014,5882.745,6693.307
3,Belgium,3908.473,4053.883,4233.076,4430.729,4580.064,4807.4,4998.998,5121.186,5315.194,5352.871,5406.979,..
4,Canada,4155.529,4228.962,4336.249,4428.753,4536.81,4635.285,5044.275,5150.470,5308.356,5189.721,5828.324,5904.939
5,Chile,1254.254,1355.452,1485.284,1658.112,1754.988,1833.552,1941.256,2119.552,2281.148,2296.988,2412.752,2607.977
6,Colombia,..,791.69,838.282,925.428,987.342,1035.02,1101.628,1139.442,1191.885,1293.363,1335.877,..
7,Costa Rica,..,1176.161,1218.357,1238.096,1312.446,1382.59,1413.267,1445.438,1529.721,1556.168,1617.823,..
8,Czech Republic,2001.958,2041.899,2090.485,2448.529,2564.609,2544.865,2670.969,2969.542,3128.523,3272.224,3805.143,..
9,Denmark,4272.376,4274.142,4433.363,4525.669,4597.459,4726.752,4900.939,5112.959,5306.704,5359.776,5693.651,6384.169
10,Estonia,1363.499,1403.665,1515.084,1636.092,1821.238,1940.411,2096.544,2201.474,2364.073,2452.289,2729.229,2989.125


In [18]:
ex_df = ex_df.melt("Country", var_name='Year', value_name='Expenditure')
ex_df['ISO3'] = ex_df.apply(lambda row: ISO3(row.Country) , axis = 1)
ex_df.Year = ex_df.Year.astype('float64')
ex_df


,Country,Year,Expenditure,ISO3
0,Australia,2010.0,3593.395,AUS
1,Austria,2010.0,4258.99,AUT
2,Belgium,2010.0,3908.473,BEL
3,Canada,2010.0,4155.529,CAN
4,Chile,2010.0,1254.254,CHL
...,...,...,...,...
451,Sweden,2021.0,6262.31,SWE
452,Switzerland,2021.0,..,CHE
453,Türkiye,2021.0,..,TUR
454,United Kingdom,2021.0,5387.23,GBR


In [19]:
le_df = pd.read_excel("WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx", skiprows=16)
le_df = le_df[['Region, subregion, country or area *', 'ISO3 Alpha-code','Year', 'Life Expectancy at Birth, both sexes (years)']]
le_df.columns = ['Country', 'ISO3', 'Year', 'Life Expectancy']

In [20]:
df = pd.merge(ex_df, le_df, on=['ISO3', 'Year'], how='inner')
df = df[['ISO3','Country_x','Year', 'Expenditure','Life Expectancy']]
df = df.rename(columns={'Country_x': 'Country'})

In [21]:
countries_with_flags = dict(zip(list(set(df.ISO3)),[name_Flag(ISO3) for ISO3 in set(df.ISO3)])) #Avoiding minutes of repeat calls
ISO3s_with_flags = dict(zip(list(set(df.ISO3)),[ISO3_flag(ISO3) for ISO3 in set(df.ISO3)]))
df['Country'] = df.apply(lambda row: countries_with_flags[row.ISO3] , axis = 1)
df['ISO3_flag']  = df.apply(lambda row: ISO3s_with_flags[row.ISO3] , axis = 1)

In [22]:
with open('expectency_expend.json', 'w') as f:
    f.write(df.to_json(orient='records'))

In [25]:
df[df.ISO3 == 'DPK']

,ISO3,Country,Year,Expenditure,Life Expectancy,ISO3_flag
